<p style="font-size: 200%; margin-top: 50px; margin-bottom: 50px;">アルゴリズム・関数集</p>

- やること
- アルゴリズムの時間計測

# 素数判定

## 試し割り法 O(N**0.5)

<ul style="line-height: 30px; margin-top: 10px; margin-bottom: 20px;">
    <li style="font-size: 125%;">2から√Nまで順に割っていき，割り切れたら素数ではない</li>
    <li style="font-size: 125%;">偶数は約数に2を持つため，省略すると少し高速化</li>
</ul>

In [1]:
# 素数ならTrue, 素数でないならFalseを返す
def is_prime(n):
    if n == 1:
        return True
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

In [3]:
is_prime(3) # 素数

True

In [4]:
is_prime(10) # 素数ではない

False

## エラトステネスの篩 O(NloglogN)

<ul style="line-height: 30px; margin-top: 10px; margin-bottom: 20px;">
    <li style="font-size: 125%;">N以下の素数を全列挙する</li>
    <li style="font-size: 125%;"><b>判定を複数行う</b>場合，前処理後はO(1)で判定可能</li>
    <li style="font-size: 125%;">デメリット：大きさNの配列を使うため，メモリ大</li>
    <li style="font-size: 125%;">手法：小さいほうから素数判定し，その数が素数ならば，その倍数をすべて削除(素数ではないため)</li>
</ul>

In [7]:
# nまでの素数判定結果リストが返される
def eratosthenes_sieve(n):
    is_prime = [True] * (n+1)
    is_prime[0] = False
    is_prime[1] = False
    for p in range(2, n+1):
        if is_prime[p]:
            for q in range(2*p, n+1, p):
                is_prime[q] = False
    return is_prime

In [8]:
# ０〜１０が素数であるか
eratosthenes_sieve(10)

[False, False, True, True, False, True, False, True, False, False, False]

In [9]:
# 使用例
n = 10
num = [2, 5, 6] # 判定用
is_prime = eratosthenes_sieve(10)
for i in num:
    print('{} : {}'.format(i, is_prime[i]))

2 : True
5 : True
6 : False


# 素因数分解

## 素因数列挙

In [2]:
def prime_factorize(n):
    a = []
    while n % 2 == 0:
        a.append(2)
        n //= 2
    f = 3
    while f * f <= n:
        if n % f == 0:
            a.append(f)
            n //= f
        else:
            f += 2
    if n != 1:
        a.append(n)
    return a

In [3]:
# 1の場合は，空リストを返す
prime_factorize(1)

[]

In [8]:
prime_factorize(280)

[2, 2, 2, 5, 7]

## 素数とその個数を取得

In [15]:
# collectionsのCounter型を使用
import collections

c = collections.Counter(prime_factorize(840))
print(c)

Counter({2: 3, 3: 1, 5: 1, 7: 1})


In [16]:
# キーと値を取り出す
k = list(c.keys())
v = list(c.values())

print(k)
print(v)

[2, 3, 5, 7]
[3, 1, 1, 1]


# 約数列挙

In [20]:
def make_divisors(n):
    lower_divisors , upper_divisors = [], []
    i = 1
    while i*i <= n:
        if n % i == 0:
            lower_divisors.append(i)
            if i != n // i:
                upper_divisors.append(n//i)
        i += 1
    return lower_divisors + upper_divisors[::-1]

In [21]:
make_divisors(10)

[1, 2, 5, 10]

# 連結成分数を求める(ABC284 - C)

## DFS

In [ ]:
# DFS
import sys
sys.setrecursionlimit(1000000)

N, M = map(int, input().split())
ans = 0  #連結成分数

# 隣接リスト
r = [[] for _ in range(N)]
for i in range(M):
    u, v = map(int, input().split())
    r[u-1].append(v-1)
    r[v-1].append(u-1)
    
# 訪問済みかどうかを管理
visited = [False] * N

# 再帰関数
def dfs(i):
    visited[i] = True
    
    for i2 in r[i]:
        if not(visited[i2]):
            dfs(i2)
            
for k in range(N):
    if visited[k]:
        continue
    else:
        ans += 1
        dfs(k)

print(ans)

## UnionFindクラス

In [ ]:
# UnionFind

from collections import defaultdict

class UnionFind():
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n

    def find(self, x):
        if self.parents[x] < 0:
            return x
        else:
            self.parents[x] = self.find(self.parents[x])
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return

        if self.parents[x] > self.parents[y]:
            x, y = y, x

        self.parents[x] += self.parents[y]
        self.parents[y] = x

    def size(self, x):
        return -self.parents[self.find(x)]

    def same(self, x, y):
        return self.find(x) == self.find(y)

    def members(self, x):
        root = self.find(x)
        return [i for i in range(self.n) if self.find(i) == root]

    def roots(self):
        return [i for i, x in enumerate(self.parents) if x < 0]

    def group_count(self):
        return len(self.roots())

    def all_group_members(self):
        group_members = defaultdict(list)
        for member in range(self.n):
            group_members[self.find(member)].append(member)
        return group_members

    def __str__(self):
        return '\n'.join(f'{r}: {m}' for r, m in self.all_group_members().items())
    
# 辺をつなげていき，最後にgroup_countを実行することで求められる

# 閉路の数を求める

## UnionFind

- uf.same(a, b)により今から繋げたい頂点同士がすでに連結成分であるかを判定
- すでに連結成分である場合は，その頂点同士を繋げることで閉路になるので，ans+=1とする

In [ ]:
# ABC288_C問題
# 閉路の数だけ，辺を削除する必要がある
# 閉路の数を，Union Findでカウント
# UnionFindクラスは，別の場所で定義済みなので，省略

N, M = map(int, input().split())
uf = UnionFind(N)
ans = 0
A, B = [], []
for i in range(M):
    a, b = map(int, input().split())
    A.append(a-1)
    B.append(b-1)

for i in range(len(A)):
    if uf.same(A[i], B[i]):
        ans += 1
    uf.union(A[i], B[i])
        
print(ans)